In [3]:
import os
import tflearn
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
import sklearn.linear_model as model
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import sklearn
from sklearn.naive_bayes import BernoulliNB
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb
from tensorflow.contrib import rnn

### Basic Logistic Regression

Use Michelle's method for importing data since I can't do it correctly:

In [5]:
train_data = np.genfromtxt('training_data.txt',dtype='str')
test_data = np.genfromtxt('test_data.txt',dtype='str')
#train_data = np.loadtxt('training_data.txt',  delimiter=' ')
#test_data = np.loadtxt('test_data.txt')
train_labels = train_data[0, :]
train_stars = train_data[1:, 0]
train_reviews = train_data[1:, 1:]

test_labels = test_data[0, :]
#test_stars = train_data[1:, 0]
test_reviews = test_data[1:, 0:]

trainX = train_reviews
trainY = train_stars
testX = test_reviews
#testY = test_stars

print("trainX = ", trainX.shape)
print("trainY = ", trainY.shape)
print("testX = ", testX.shape)


for i in range(0, len(trainX)):
    for j in range(0, len(trainX[0])):
        trainX[i][j] = int(trainX[i][j])

for i in range(0, len(testX)):
    for j in range(0, len(testX[0])):
        testX[i][j] = int(testX[i][j])

for i in range(0, len(trainY)):
        trainY[i] = int(trainY[i])
#for i in range(0, len(trainY)):
        #testY[i] = int(testY[i])

trainX =  (20000, 1000)
trainY =  (20000,)
testX =  (10000, 1000)


Define a function for classification error:

In [6]:
def class_error(y, y_pred):
    misclassified = 0
    for i in range(len(y)):
        if y[i] != y_pred[i]:
            misclassified += 1
    return float(misclassified) / len(y)

Split data into 80% training set and 20% test set.

In [7]:
num_rows = []
for i in range(len(trainY)):
    num_rows.append(i)
    
np.random.shuffle(num_rows)
train_indices = num_rows[:16000]
valid_indices = num_rows[16000:]

train_X = []
train_y = []
valid_X = []
valid_y = []

for index in train_indices:
    train_X.append(trainX[index])
    train_y.append(trainY[index])
    
for index in valid_indices:
    valid_X.append(trainX[index])
    valid_y.append(trainY[index])

With Michelle's code, it doesn't actually convert the array contents into integers because she tried to do it in place. This code does it correctly out of place:

In [8]:
train_X2 = []
for row in train_X:
    temp = []
    for element in row:
        temp.append(int(element))
    train_X2.append(temp)

valid_X2 = []
for row in valid_X:
    temp = []
    for element in row:
        temp.append(int(element))
    valid_X2.append(temp)
    
valid_y2 = []
for row in valid_y:
    valid_y2.append(int(row))
    
train_y2 = []
for row in train_y:
    train_y2.append(int(row))
    
test_X2 = []
for row in testX:
    temp = []
    for element in row:
        temp.append(int(element))
    test_X2.append(temp)

Convert everything back to a numpy array:

In [ ]:
trainX = np.array(train_X2)
validX = np.array(valid_X2)
trainY = np.array(train_y2)
validY = np.array(valid_y2)
testX = np.array(test_X2)

# Recurrent Neural Network 

In [ ]:
#trainY_1D = []
#for i in range(0, len(trainY)):
    #if trainY[i][0]==1:
        #trainY_1D.append(0)
    #else:
        #trainY_1D.append(1)
        
#trainY = trainY_1D

#print(trainY_1D)

In [ ]:
print(validY)

In [ ]:
#trainY = to_categorical(trainY, 2)

net = tflearn.input_data([None, 1000])
print("made it here 1")
net = tflearn.embedding(net, input_dim=20000, output_dim=128)
print("made it here 2")
net = tflearn.lstm(net, 128, dropout=0.8)
print("made it here 3")
net = tflearn.fully_connected(net, 2, activation='softmax')
print("made it here 4")
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')
print("made it here 5")

In [ ]:
# Training
model = tflearn.DNN(net, clip_gradients=0., tensorboard_verbose=2)
print("made it here 6")
model.fit(trainX, trainY, validation_set =(validX, validY), show_metric=True, batch_size=64)
print("made it here 7")

Fit the model:

In [ ]:
logistic = model.LogisticRegression()
logistic.fit(train_X2, train_y2)

Print the classification error on the training set:

In [ ]:
class_error(logistic.predict(train_X2), train_y2)

Print the classification error on the validation set:

In [ ]:
class_error(logistic.predict(valid_X2), valid_y2)

Logistic regression works better than a random forest. Now, we want to combine the training and validation sets again so that we can train our model on all the data to prevent overfitting. We will then use this to predict on the test set:

In [ ]:
all_X = np.concatenate((train_X2, valid_X2))
all_y = np.concatenate((train_y2, valid_y2))

In [ ]:
logistic = model.LogisticRegression()
logistic.fit(all_X, all_y)

A function for writing to the text file for the submission:

In [ ]:
def writeToText(predictions, filename):
    array = [["Id","Prediction"]]
    index = 1
    for i in range(len(predictions)):
        predict = predictions[index - 1]
        array.append([index, int(predictions[index - 1])])
        index += 1
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter=',', quotechar='|')
    writer.writerows(array)

In [ ]:
writeToText(logistic.predict(test_X2), 'logistic.txt')

### Logistic Regression with TF-IDF

A function to convert the set into a TF-IDF matrix:

In [ ]:
def TFIDF(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element)):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    tokenize = lambda doc: doc.lower().split(" ")
    sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
    return sklearn_tfidf.fit_transform(documents)

Fit the model using the training data:

In [ ]:
logistic = model.LogisticRegression()
logistic.fit(TFIDF(train_X2, train_labels), train_y2)

In [ ]:
class_error(logistic.predict(TFIDF(valid_X2, train_labels)), valid_y2)

This model has a lower classification error than the basic regression. We will now undergo the same procedure as in the last section to get the test predictions:

In [ ]:
logistic = model.LogisticRegression()
logistic.fit(TFIDF(all_X, train_labels), all_y)
writeToText(logistic.predict(TFIDF(test_X2, train_labels)), 'logisticTFIDF.txt')

### GridSearch

In [ ]:
clf = SGDClassifier(loss='hinge', penalty='l2', random_state=42).fit(train_X2, train_y2)
parameters = {'alpha': (1e-5, 1e-6)}
gs_clf = GridSearchCV(clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(TFIDF(train_X2, train_labels), train_y2)

In [ ]:
gs_clf.best_params_

clf.get_params()

### Naive Bayes

In [ ]:
nb = MultinomialNB(alpha=50).fit(TFIDF(train_X2, train_labels), train_y2)

In [ ]:
nb.get_params()

In [ ]:
class_error(nb.predict(TFIDF(valid_X2, train_labels)), valid_y2)

### SVM

In [ ]:
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(TFIDF(train_X2, train_labels), train_y2)

In [ ]:
class_error(sgd.predict(TFIDF(valid_X2, train_labels)), valid_y2)

In [ ]:
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42)
sgd.fit(TFIDF(all_X, train_labels), all_y)
writeToText(sgd.predict(TFIDF(test_X2, train_labels)), 'SGD_TFIDF.txt')

### Blend the based on equal weights for each one:

In [ ]:
valid_TFIDF = TFIDF(valid_X2, train_labels)
train_TFIDF = TFIDF(train_X2, train_labels)
all_TFIDF = TFIDF(all_X, train_labels)
test_TFIDF = TFIDF(test_X2, train_labels)

In [ ]:
logistic = model.LogisticRegression()
logistic.fit(train_TFIDF, train_y2)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(train_TFIDF, train_y2)

nb = MultinomialNB(alpha=50).fit(train_TFIDF, train_y2)

pred1 = logistic.predict(valid_TFIDF)
pred2 = sgd.predict(valid_TFIDF)
pred3 = nb.predict(valid_TFIDF)

In [ ]:
pred = []
for i in range(len(pred1)):
    
    if pred1[i] == pred2[i]:
        pred.append(pred1[i])
    elif pred1[i] == pred3[i]:
        pred.append(pred1[i])
    else:
        pred.append(pred2[i])

In [ ]:
class_error(pred, valid_y2)

In [ ]:
logistic = model.LogisticRegression()
logistic.fit(all_TFIDF, all_y)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(all_TFIDF, all_y)

nb = MultinomialNB(alpha=50).fit(all_TFIDF, all_y)

pred1 = logistic.predict(test_TFIDF)
pred2 = sgd.predict(test_TFIDF)
pred3 = nb.predict(test_TFIDF)

pred = []
for i in range(len(pred1)):
    
    if pred1[i] == pred2[i]:
        pred.append(pred1[i])
    elif pred1[i] == pred3[i]:
        pred.append(pred1[i])
    else:
        pred.append(pred2[i])

writeToText(pred, 'Blend_TFIDF.txt')

### BernoulliNB

For this, we need binary features.

In [ ]:
train_X2[train_X2 > 1] = 1
valid_X2[valid_X2 > 1] = 1
all_X[all_X > 1] = 1

In [ ]:
bnb = BernoulliNB(alpha=1.0e-10)
bnb.fit(train_X2, train_y2)
class_error(bnb.predict(valid_X2), valid_y2)

### Probability Blend

Each model outputs can output a probability rather than a 0 or 1. We should use these probabilities when classifying the points.

In [ ]:
logistic = model.LogisticRegression()
logistic.fit(train_TFIDF, train_y2)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(train_TFIDF, train_y2)

nb = MultinomialNB(alpha=50).fit(train_TFIDF, train_y2)

In [ ]:
nb.predict_proba(valid_TFIDF)

In [ ]:
sgd.decision_function(valid_TFIDF)

In [ ]:
valid_y2

In [ ]:
logistic.decision_function(valid_TFIDF)

sgd and logistic have decision functions, whereas bayes has a probability. Since sgd and logistic are better models, we will look at when the magnitude of their confidence scores is greater than 1. If they both have magnitudes greater than 1 but opposite signs, or if the magnitude for both is less than 1, we get the majority vote from all 3 models.

In [ ]:
def tieBreak(i, pred1, pred2, pred3):
    winner = 0
    if pred1[i] == pred2[i]:
        winner = pred1[i]
    elif pred1[i] == pred3[i]:
        winner = pred1[i]
    else:
        winner = pred2[i]
    return winner

In [ ]:
sgd_confidence = sgd.decision_function(valid_TFIDF)
log_confidence = logistic.decision_function(valid_TFIDF)
pred1 = sgd.predict(valid_TFIDF)
pred2 = logistic.predict(valid_TFIDF)
pred3 = nb.predict(valid_TFIDF)

pred_blend = []
for i in range(len(pred1)):
    if abs(sgd_confidence[i]) > 1 and abs(log_confidence[i]) < 1:
        pred_blend.append(pred1[i])
    elif abs(log_confidence[i]) > 1 and abs(sgd_confidence[i]) < 1:
        pred_blend.append(pred2[i])
    elif abs(log_confidence[i]) > 1 and abs(sgd_confidence[i]) > 1:
        if (log_confidence[i] > 0 and sgd_confidence[i] > 0) or (log_confidence[i] < 0 and sgd_confidence[i] < 0):
            pred_blend.append(pred1[i])
        else:
            pred_blend.append(tieBreak(i, pred1, pred2, pred3))
    else:
        pred_blend.append(tieBreak(i, pred1, pred2, pred3))
        print(sgd_confidence[i])
        print(log_confidence[i])
        print(valid_y2[i])

In [ ]:
class_error(pred_blend, valid_y2)